<a href="https://colab.research.google.com/github/jocelynbaduria/cmpe-297_SOTA/blob/main/Transformer_MLM_and__NSP_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReadMe :
Reference : 

https://github.com/jamescalam/transformers/blob/main/course/training/08_mlm_and_nsp_training.ipynb

https://www.youtube.com/watch?v=1gN1snKBLP0

1. Install some module gradio app and weights and biases for experiment tracking.

2. Reproduce the code for understanding of Transformer MLM and NSP.

    a. Import module torch, Bert tokenizer, Bert for Next Sentence Prediction and Weights and Biases.

    b. Modify data into mix of non-random sentences, and random sentences for NSP. 

    c. Creates a 'bag of random sentences that we can pull from when selecting a random sentence B.

    d. Create a 50/50 NSP training data.

    e. Tokenize the data using a truncation/padding max_length of 512.
       This results to out-of-memory using GPU. Change the max_length=256.
       inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=256, truncation=True, padding='max_length')

    f. Create a labels tensor for MLM, and the next_sentence_label for NSP. The next_sentence_label tensor is simply a LongTensor([label]).T 
    Labels tensors is simply a clone of the input_ids tensor before masking.

    g. Now mask the tokens in the input_ids tensor using the 15% probability for MLM - ensuring we don't mask CLS, SEP, or PAD tokens. Then take the indices of each True value within each vector. 

    h. Apply these indices to each row in input_ids, assigning each value at these indices a value of 103. 

    i. Now all inputs and labels are ready, so set up the inputs to be fed into model during training. Create a PyTorch dataset from our data.

    j. Initialize the data using the MeditationsDataset class and Dataloader to load the data into model training. 

    k. Setup the training loop setting and the GPU/CPU usage. Activate the training mode of our model, and initialize the optimizer (Adam with weighted decay - reduces chance of overfitting).

    l. Initialize the weights and biases for training 2 epochs. 
       Result : 
       Activation for is the next sentence -1.1045 and not the next sentence 2.5738. 

       output.logits
          next sentence  not next sentence 
       tensor([[-1.1045,  2.5738]], device='cuda:0', grad_fn=<AddmmBackward>)

    m. Do the prediction
       Prediction is 1 means the next sentence(relationship with sentence_a to sentence_b)

       torch.argmax(outputs.logits)
       tensor(1, device='cuda:0')

    n. Add the gradio app for testing. I created the function for showing the results of next sentence based on the model train of MLM_NSP.



1. Install some module gradio app and weights and biases for experiment tracking.

In [ ]:
!pip install gradio -q
# Install wandb for experiment tracking
!pip install --upgrade https://github.com/wandb/client/archive/feature/code-save.zip -q
!pip install transformers -q

     |████████████████████████████████| 3.6 MB 5.3 MB/s 
     |████████████████████████████████| 206 kB 46.5 MB/s 
     |████████████████████████████████| 1.9 MB 32.9 MB/s 
     |████████████████████████████████| 3.5 MB 34.9 MB/s 
     |████████████████████████████████| 961 kB 38.2 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     - 10.4 MB 1.2 MB/s
     |████████████████████████████████| 180 kB 5.3 MB/s 
     |████████████████████████████████| 76 kB 4.4 MB/s 
     |████████████████████████████████| 139 kB 43.5 MB/s 
     |████████████████████████████████| 97 kB 4.9 MB/s 
     |████████████████████████████████| 70 kB 7.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 
     |████████████████████████████████| 596 kB 31.8 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 34.3 MB/s 


2. Reproduce the code for understanding of Transformer MLM and NSP.

In [ ]:
# from transformers import BertTokenizer, BertForPreTraining
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForPreTraining.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

with open('/content/drive/MyDrive/cmpe-297-SOTA/data/text/meditations/clean.txt', 'r') as fp:
    text = fp.read().split('\n')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Other imports 
import wandb
wandb.login()
from wandb.keras import WandbCallback
from tqdm import tqdm

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# text[:3]
text[:10]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander.",
 'From Diognetus, not to busy myself about trifling things, and not to give credit to what 

Modify data into mix of non-random sentences, and random sentences for NSP.
Creates a 'bag of random sentences that we can pull from when selecting a random sentence B.

In [ ]:
bag = [item for sentence in text for item in sentence.split('.') if item != '']
bag_size = len(bag)

Create a 50/50 NSP training data

In [ ]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [ sentence for sentence in paragraph.split('.') if sentence != '']

    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 wether isNextSentence or Not NextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence 
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this no NotNetSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

Tokenize the data using a truncation/padding max_length of 512

In [ ]:
# inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length') - out of memeory
# inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=None, truncation=True, padding='max_length') - works 
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=256, truncation=True, padding='max_length')

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
inputs

{'input_ids': tensor([[  101,  2002,  2001,  ...,     0,     0,     0],
        [  101,  2057,  2113,  ...,     0,     0,     0],
        [  101,  2582,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Create a labels tensor for MLM, and the next_sentence_label for NSP. The next_sentence_label tensor is simply a LongTensor([label]).T 

In [ ]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T

In [ ]:
inputs['next_sentence_label']

tensor([[0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],

Labels tensors is simply a clone of the input_ids tensor before masking

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs['labels']

tensor([[  101,  2002,  2001,  ...,     0,     0,     0],
        [  101,  2057,  2113,  ...,     0,     0,     0],
        [  101,  2582,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]])

Now we mask tokens in the input_ids tensor using the 15% probability for MLM - ensuring we don't mask CLS, SEP, or PAD tokens.

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)

In [ ]:
# create mask array
mask_array = (rand < 0.15) * (inputs.input_ids != 101) * \
              (inputs.input_ids != 102) * (inputs.input_ids != 0)

Take the indices of each True value within each vector

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_array[i].nonzero()).tolist()
    )

In [ ]:
selection[:4]

[[10, 12, 13, 22, 32, 37, 44, 45, 47, 49, 62, 66, 71, 74, 75],
 [9, 30, 43],
 [6,
  15,
  49,
  50,
  58,
  62,
  90,
  96,
  112,
  127,
  133,
  136,
  140,
  144,
  156,
  159,
  169,
  174,
  197,
  205,
  214,
  223,
  229,
  230],
 [8,
  14,
  21,
  23,
  28,
  29,
  31,
  35,
  39,
  40,
  47,
  58,
  65,
  74,
  91,
  107,
  110,
  119,
  134,
  137,
  147]]

Apply these indices to each row in input_ids, assigning each value at these indices a value of 103.

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

Now all inputs and labels are ready, so we can begin setting up inputs to be fed into model during training. Create a PyTorch dataset from our data.

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

Initialize the data using the MeditationsDataset class


In [ ]:
dataset = MeditationsDataset(inputs)

Initialize the Dataloader to load the data into model training


In [ ]:
# loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

Setup the training loop setting and the GPU/CPU usage

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move the model to the selected device
model.to(device)

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

Torch 1.9.0+cu111 CUDA 11.1
Device: cuda:0


Activate the training mode of our model, and initialize the optimizer (Adam with weighted decay - reduces chance of overfitting).

In [ ]:
from transformers import AdamW
# activate training mode
model.train()

# intialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

from tqdm.notebook import tqdm # for progress bar

In [ ]:
# Initialize the weights and biases
wandb.init(project="mlm_nsp", id="bert_pretrained_batchsize1_maxlength256_nsp")
from tqdm.notebook import tqdm # for progress bar

epochs = 2

for epoch in range(epochs):
  # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels, callbacks=[WandbCallback()])
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

wandb: Wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/159 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
/usr/local/lib/python3.7/dist-packages/transformers/models/bert/modeling_bert.py:1440: FutureWarning: The `next_sentence_label` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,
Streaming file created twice in same run: /content/wandb/run-20211023_033158-bert_pretrained_batchsize1_maxlength256_nsp/wandb-events.jsonl


  0%|          | 0/159 [00:00<?, ?it/s]

In [ ]:
model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels, callbacks=[WandbCallback()])

/usr/local/lib/python3.7/dist-packages/transformers/models/bert/modeling_bert.py:1440: FutureWarning: The `next_sentence_label` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


NextSentencePredictorOutput([('loss',
                              tensor(0.0437, device='cuda:0', grad_fn=<NllLossBackward>)),
                             ('logits',
                              tensor([[-0.9580,  2.1501]], device='cuda:0', grad_fn=<AddmmBackward>))])

In [ ]:
# activation for is the next sentence -1.1045 and not the next sentence 2.5738.
outputs.logits

tensor([[-1.1045,  2.5738]], device='cuda:0', grad_fn=<AddmmBackward>)

In [ ]:
# Prediction is 1 means is the next sentence.
torch.argmax(outputs.logits)

tensor(1, device='cuda:0')

Do the training loop for two epochs(you can change epochs for modificatiions)

Add the gradio app for testing


In [ ]:
import gradio as gr
# from transformers import BertTokenizer, BertForPreTraining
from transformers import BertTokenizer, BertForNextSentencePrediction

import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

model.train()

def next_sentence_prediction(inputs):
    input_ids = batch['input_ids'].to(device)
    # input_ids = tokenizer.encode(batch['inputs_ids'], return_tensors='pt')
    outputs = model.generate(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label, max_length=100, num_beams=5,
                        labels=labels, callbacks=[WandbCallback()])
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    next_sentence_label = batch['next_sentence_label'].to(device)
    labels = batch['labels'].to(device)
    # outputs = model.generate(input_ids, max_length=100, num_beams=5,
    #       no_repeat_ngram_size=2, early_stopping=True)
    # output = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    output = outputs.logits[0]
    return ".".join(output.split(".")[:-1]) + "."


output_text = gr.outputs.Textbox(label=True)
gr.Interface(next_sentence_prediction,"textbox", output_text, title="Next Sentence Prediction", description="Meditation_Text").launch()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://39901.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7875/',
 'https://39901.gradio.app')